# dunedata.ipynb
This notebook uses dunerun to search for some DUNE data. 

## Introduction

First connect to a Jupyter machine where DUNE cvmfs is available e.g. <https://analytics-hub.fnal.gov>, open a terminal and and follow the [instructions](https://github.com/dladams/dunesw-support#readme) to install *dunerun*.
Here we assume the package is installed at ~/proc/install/common/dunerun but any location is fine.  

Copy this noteboook dunerun.ipynb from the installation area to a directory of your choosing and open the copy on Jupyter.
Since you are reading this, you may have already done so.  

Run the *dunerun* setup (change the location to match your installation) and import dunerun and other packages of interest:

In [1]:
%run ~/proc/install/common/dunerun/python/setup.py
import sys
import os
import dunerun
print(f"dunerun version is {dunerun.version()}")
dsw = None
check_proxy = True                  # Set this False to skip the proxy check and generation
have_nopassword_kinit = False       # Set this True if kinit w/o password is enabled
def check_proxy(doprint):           # Function to check if user has a proxy. Function is defined because we do this twice.
    need_proxy = False
    if doprint: print(f">>>>> Checking VOMS proxy.")
    voms_out = os.system('voms-proxy-info -exists -valid 0:5')
    stat = int(voms_out)
    return stat != 0

dunerun version is 1.0.6


## List releases

Uncomment the following if you want to see a list of available versions of dunesw.

In [2]:
#dunerun.DuneRun('dune').run('ups list -aK+ dunesw')

## VOMS proxy

Access to DUNE data requires a VOMS proxy which can be otained with kerberos credentials. The following lines run system commands to check if we have such credentials and such a proxy. Not use of the Ipython magic '!' to run a system command and capture the ouput.  

The kx509 package used in generating the proxy is not installed on the FNAL analysis platform and so we used *DuneRun* with the *dune* environment to set up that package from cvmfs.

The following block generates the credentials and proxy if they are needed. The 'Setup AC' warning is annoying but harmless. it and the need to set up the kx509 product are because kx509 is not installed.

In [3]:
linesep = '-------------------------------------------------------------'
need_cred = False
need_proxy = False
if check_proxy:
    !date
    print(linesep)
    need_proxy = check_proxy(True)
    if need_proxy:
        print('>>>>> ERROR: VOMS proxy not found.')
        print(f"{linesep}\n>>>>> Checking kerberos.")
        klist_out = !klist
        for line in klist_out:
            print(line)
        if 'No credentials' in klist_out[0]:
            print('>>>>> ERROR: Kerberos credentials not found.')
            if not have_nopassword_kinit:
                raise Exception('>>>>> Run kinit in a terminal and restart notebook.')
            need_cred = True
    print(linesep)
else:
    print('Skipped proxy check')

Thu Mar 17 13:01:22 UTC 2022
-------------------------------------------------------------
>>>>> Checking VOMS proxy.
-------------------------------------------------------------


In [4]:
if need_cred:
    if have_nopassword_kinit:
        !kinit
        need_cred = False
    else:
        raise Exception('>>>>> Run kinit in a terminal and restart notebook.')
if need_proxy:
    dune = dunerun.DuneRun('dune', shell=True)
    dune.run('setup kx509')
    dune.run('kx509 --minhours 12')
    dune.run('voms-proxy-init -noregen -rfc -voms dune:/dune/Role=Analysis')
    if check_proxy(False): raise Exception('Unable to obtain proxy.')
    print(line)

## Set up a release

Use *DuneRun* to start a process using a DUNE release.

In [5]:
if dsw is None:
    dsw = dunerun.DuneRun('dunesw', 'v09_45_00_00:e20:prof', shell=True)
    dsw.run('duneHelp')

Setting up dunesw v09_45_00_00 e20:prof


Welcome to dunetpc 
Some available commands:
              duneHelp - Display information about the current setup of dunetpc.
                   lar - Run the art/larsoft event looop e.g. to process event data.
  product_sizes_dumper - Display the products and size in an event data file.
               fcldump - Display the resolved configuration for a fcl file.
               liblist - List available plugin libraries.
        pdChannelRange - Display protoDUNE channel grops and ranges.
           duneRunData - Display run data for a run.
           duneTestFcl - Test some high-level fcl configurations.
Use option "-h" with any of these for more information.


## Finding data

The samweb command can be used to find DUNE data. If enabled, the first block shows some help messages.  

The next blocks showing how to list all the raw data files for a protDUNE run, find the URL(s) for one of those files and then open the file with Root.

In [6]:
if False:
    dsw.run('samweb --help-commands')
    print(linesep)
    dsw.run('samweb list-files -h')
    print(linesep)
    dsw.run('samweb list-files --help-dimensions')
    print(linesep)

In [7]:
run = 5240
query = f"data_tier raw and DUNE_data.is_fake_data 0 and run_number {run}"
if True:
    print(f">>>>> Raw data files for run {run}")
    print(f">>>>> Query: {query}")
    dsw.run(f'samweb list-files "{query}"')

>>>>> Raw data files for run 5240
>>>>> Query: data_tier raw and DUNE_data.is_fake_data 0 and run_number 5240
np04_raw_run005240_0003_dl5.root
np04_raw_run005240_0004_dl7.root
np04_raw_run005240_0004_dl3.root
np04_raw_run005240_0003_dl11.root
np04_raw_run005240_0004_dl12.root
np04_raw_run005240_0004_dl10.root
np04_raw_run005240_0003_dl6.root
np04_raw_run005240_0001_dl7.root
np04_raw_run005240_0001_dl12.root
np04_raw_run005240_0003_dl7.root
np04_raw_run005240_0003_dl4.root
np04_raw_run005240_0003_dl12.root
np04_raw_run005240_0003_dl3.root
np04_raw_run005240_0003_dl1.root
np04_raw_run005240_0003_dl9.root
np04_raw_run005240_0003_dl10.root
np04_raw_run005240_0003_dl2.root
np04_raw_run005240_0003_dl8.root
np04_raw_run005240_0002_dl1.root
np04_raw_run005240_0002_dl4.root
np04_raw_run005240_0002_dl12.root
np04_raw_run005240_0002_dl3.root
np04_raw_run005240_0002_dl2.root
np04_raw_run005240_0002_dl11.root
np04_raw_run005240_0002_dl6.root
np04_raw_run005240_0002_dl9.root
np04_raw_run005240_0002_

In [8]:
fnam = 'np04_raw_run005240_0001_dl1.root'
dsw.run(f"samweb get-file-access-url {fnam} --schema=root")

root://castorpublic.cern.ch//castor/cern.ch/neutplatform/protodune/rawdata/np04/detector/None/raw/06/68/39/48/np04_raw_run005240_0001_dl1.root
root://fndca1.fnal.gov:1094/pnfs/fnal.gov/usr/dune/tape_backed/dunepro/protodune/np04/beam/detector/None/raw/06/68/39/48/np04_raw_run005240_0001_dl1.root


## Reading data

Finally we read the file to check that we really have access. First open the file in root and then run the art utility that counts the events in the file.

In [9]:
furl = 'root://fndca1.fnal.gov:1094/pnfs/fnal.gov/usr/dune/tape_backed/dunepro/protodune/np04/beam/detector/None/raw/06/68/39/48/np04_raw_run005240_0001_dl1.root'
print('>>>>> Opening file with Root.')
dsw.run(f"root.exe -q {furl}")

>>>>> Opening file with Root.
   ------------------------------------------------------------------
  | Welcome to ROOT 6.22/08                        https://root.cern |
  | (c) 1995-2020, The ROOT Team; conception: R. Brun, F. Rademakers |
  | Built for linuxx8664gcc on Mar 10 2021, 14:20:04                 |
  | From tags/v6-22-08@v6-22-08                                      |
  | Try '.help', '.demo', '.license', '.credits', '.quit'/'.q'       |
   ------------------------------------------------------------------


Attaching file root://fndca1.fnal.gov/pnfs/fnal.gov/usr/dune/tape_backed/dunepro/protodune/np04/beam/detector/None/raw/06/68/39/48/np04_raw_run005240_0001_dl1.root as _file0...
(TFile *) 0x46c8bd0


In [10]:
print('>>>>> Counting events.')
dsw.run(f"count_events --hr {furl}")

>>>>> Counting events.
root://fndca1.fnal.gov:1094/pnfs/fnal.gov/usr/dune/tape_backed/dunepro/protodune/np04/beam/detector/None/raw/06/68/39/48/np04_raw_run005240_0001_dl1.root	1 run, 1 subrun, 107 events, and 0 results.
Counted events successfully for 1 specified files.
